In [1]:
using IJulia
using NBInclude
include("IntegralUtils.jl")
using .IntegralUtils

In [2]:
"""
    Φ(F::Function, X::Tuple{Number, Number}, Y::Tuple{Number, Number}, Z::Tuple{Number, Number}; ϵ::Number = 1e-3, n::Int = 1)::Union{::Int, ::Float64}

Determine a flux of a vector field `F` through a cuboid.

# Examples
```
julia> Φ((x, y, z) -> [x, y, z], (0, 1), (0, 1), (0, 1))
2.9999999999958114

julia> Φ((x, y, z) -> [-y, x, 0], (-1, 1), (-1, 1), (-1, 1))
0.0
```
"""
function Φ(F::Function, X::Tuple{Number, Number}, Y::Tuple{Number, Number}, Z::Tuple{Number, Number}; ϵ::Number = 1e-3, n::Int = 1)::Float64
    Φₙ = surface_integral(F, X, Y, Z; ξ = 2n, υ = 2n, ζ = 2n)
    n += 1
    Φₙ₊₁ = surface_integral(F, X, Y, Z; ξ = 2n, υ = 2n, ζ = 2n)
    while abs(Φₙ₊₁ - Φₙ) > ϵ
        n += 1
        Φₙ, Φₙ₊₁ = Φₙ₊₁, surface_integral(F, X, Y, Z; ξ = 2n, υ = 2n, ζ = 2n)
    end
    return round_float(Φₙ₊₁, ϵ)
end

Φ

In [3]:
"""
    Φ(F::Function, r::Function, U::Tuple{Number, Number}, V::Tuple{Number, Number}; ϵ::Number = 1e-3, n::Int = 1)::Union{::Int, Float64}

Determine a flux of a vector field `F` through a surface parametrized by `r`.

# Examples
```
julia> Φ((x, y, z) -> [-y, x, 0], (u, v) -> [(cos(u) + 2)cos(v), (cos(u) + 2)sin(v), sin(u)],  (0, 2π), (0, π/2))
-9.869203248214035e-6

julia> Φ((x, y, z) -> [x, y, z], (u, v) -> [cos(u)cos(v), sin(u)cos(v), sin(v)], (0, 2π), (-π/2, π/2))
12.566373879290955
```
"""
function Φ(F::Function, r::Function, U::Tuple{Number, Number}, V::Tuple{Number, Number}; ϵ::Number = 1e-3, n::Int = 1)#::Float64
    Φₙ = surface_integral(F, r, U, V; μ = 2n, ν = 2n)
    n += 1
    Φₙ₊₁ = surface_integral(F, r, U, V; μ = 2n, ν = 2n)
    while abs(Φₙ₊₁ - Φₙ) > ϵ
        n += 1
        Φₙ, Φₙ₊₁ = Φₙ₊₁, surface_integral(F, r, U, V; μ = 2n, ν = 2n)
    end
    return round_float(Φₙ₊₁, ϵ)
end

Φ

In [8]:
?Φ

"Φ" can be typed by \Phi<tab>

search: Φ



```
Φ(F::Function, X::Tuple{Number, Number}, Y::Tuple{Number, Number}, Z::Tuple{Number, Number}; ϵ::Number = 1e-3, n::Int = 1)::Float64
```

Determine a flux of a vector field `F` through a cuboid.

# Examples

```
julia> Φ((x, y, z) -> [x, y, z], (0, 1), (0, 1), (0, 1))
2.9999999999958114

julia> Φ((x, y, z) -> [-y, x, 0], (-1, 1), (-1, 1), (-1, 1))
0.0
```

---

```
Φ(F::Function, r::Function, U::Tuple{Number, Number}, V::Tuple{Number, Number}; ϵ::Number = 1e-3, n::Int = 1)::Float64
```

Determine a flux of a vector field `F` through a surface parametrized by `r`.

# Examples

```
julia> Φ((x, y, z) -> [-y, x, 0], (u, v) -> [(cos(u) + 2)cos(v), (cos(u) + 2)sin(v), sin(u)],  (0, 2π), (0, π/2))
-9.869203248214035e-6

julia> Φ((x, y, z) -> [x, y, z], (u, v) -> [cos(u)cos(v), sin(u)cos(v), sin(v)], (0, 2π), (-π/2, π/2))
12.566373879290955
```


In [5]:
Φ((x, y, z) -> [-y, x, 0], (u, v) -> [(cos(u) + 2)cos(v), (cos(u) + 2)sin(v), sin(u)],  (0, 2π), (0, π/2))

0

In [15]:
function simpson(f::Function, a::Number, b::Number, n::Number)
    n % 2 == 0 || error("`n` must be even")
    h = (b - a) / n
    s = f(a) + f(b)
    s += 4sum(f.(a .+ collect(1:2:n) .* h))  # even x's
    s += 2sum(f.(a .+ collect(2:2:n-1) .* h))  # odd x's
    return h / 3 * s
end

simpson (generic function with 1 method)

In [16]:
simpson(x -> x*sin(x), 0, pi/2, 4)  # 1

0.9995912331140006

In [8]:
function double_simpson(f::Function, ax::Number, bx::Number,
                        ay::Number, by::Number, nx::Number, ny::Number)

    hx = (bx - ax) / nx
    hy = (by - ay) / ny
    s = 0
    for i in 0:ny
        if i == 0 | i == ny
            p = 1
        elseif i % 2 != 0
            p = 4
        else
            p = 2
        end
        for j in 0:nx
            if j == 0 | j == nx
                q = 1
            elseif j % 2 != 0
                q = 4
            else
                q = 2
            end
            x = ax + j*hx
            y = ay + i*hy
            s += p * q * f(x, y)
        end
    end
    return s * hx * hy / 9
end



double_simpson (generic function with 1 method)

In [9]:
double_simpson((x, y)->x^2*y + x*y^2, 1, 2, -1, 1, 80, 80) |> println # 1
double_simpson((x, y)->cos(x + y) + y*sin(x +y), -π/2, π, 0, π, 800, 800) |> println  # π - 4 = -0.8594

0.9958333333333333
-0.8492443678334637


In [14]:
# próbuję zbroadcastować double simpsona, ale już na tym etapie wynik jest minimalnie inny niż ten wyżej
# jeszcze nie znalazłem przyczyny
# Marcin M

function double_simpson2(f::Function, ax::Number, bx::Number,
                        ay::Number, by::Number, nx::Number, ny::Number)

    hx = (bx - ax) / nx
    hy = (by - ay) / ny
    s = 0
    for i in 0:ny
        if i == 0 | i == ny
            p = 1
        elseif i % 2 != 0
            p = 4
        else
            p = 2
        end
        s += p * 4sum(f.((ax .+ collect(1:2:nx) .* hx), ay + i*hy))
        s += p * 2sum(f.((ax .+ collect(2:2:nx-1) .* hx), ay + i*hy))
        s += p * f(ax, ay + i*hy) + p * f(ax + nx*hx, ay + i*hy)
    end
    return s * hx * hy / 9
end

double_simpson2 (generic function with 1 method)

In [13]:
double_simpson2((x, y)->x^2*y + x*y^2, 1, 2, -1, 1, 80, 80) |> println # 1
double_simpson2((x, y)->cos(x + y) + y*sin(x +y), -π/2, π, 0, π, 800, 800) |> println  # π - 4 = -0.8594

0.9712448133680562
-0.8322294318588972


## Marcina K wersja

In [17]:
# this function creates an array with weights for double simpson
function create_array(nx::Number, ny::Number)
    (nx % 2 == 0 & ny % 2 == 0) || error("both nx and ny must be even numbers")
    w = repeat([2], nx+1); w[1] = 1; w[nx+1] = 1
    z = repeat([2], ny+1); z[1] = 1; z[ny+1] = 1
    w[2:2:nx] .= 4
    z[2:2:ny] .= 4
    return w * z'
end

create_array (generic function with 1 method)

In [18]:
# example
create_array(10, 8)

11×9 Array{Int64,2}:
 1   4  2   4  2   4  2   4  1
 4  16  8  16  8  16  8  16  4
 2   8  4   8  4   8  4   8  2
 4  16  8  16  8  16  8  16  4
 2   8  4   8  4   8  4   8  2
 4  16  8  16  8  16  8  16  4
 2   8  4   8  4   8  4   8  2
 4  16  8  16  8  16  8  16  4
 2   8  4   8  4   8  4   8  2
 4  16  8  16  8  16  8  16  4
 1   4  2   4  2   4  2   4  1

In [19]:
function doubleSimpson(f::Function, ax::Number, bx::Number,
                        ay::Number, by::Number, nx::Number, ny::Number)
    weights = create_array(nx, ny)
    nums = zeros(ny+1, nx+1)
    hx = (bx - ax) / nx
    hy = (by - ay) / ny
    for i in 1:ny+1
        y = ay + hy * (i-1)
        for j in 1:nx+1
            x = ax + hx * (j-1)
            nums[i, j] = f(x, y)
        end
    end
    return sum(weights .* nums) * hx * hy / 9  
end

doubleSimpson (generic function with 1 method)

In [20]:
doubleSimpson((x, y)->x^2*y + x*y^2, 1, 2, -1, 1, 80, 80) |> println # 1
doubleSimpson((x, y)->cos(x + y) + y*sin(x +y), -π/2, π, 0, π, 800, 800) |> println  # π - 4 = -0.8594

1.0000000000000007
-0.8584073464065138


MethodError: MethodError: no method matching copyto!(::Tuple{Int64,Int64,Int64}, ::Base.Broadcast.Broadcasted{Base.Broadcast.Style{Tuple},Tuple{Base.OneTo{Int64}},typeof(+),Tuple{Tuple{Int64,Int64,Int64},Int64}})
Closest candidates are:
  copyto!(!Matched::AbstractArray, ::Base.Broadcast.Broadcasted) at broadcast.jl:863
  copyto!(!Matched::AbstractArray, ::Any) at abstractarray.jl:720